In [548]:
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import osmnx as ox
import pandas as pd
import pickle
from pathlib import Path
from tqdm import tqdm
from matplotlib.colors import Normalize
import requests
import zipfile
import os

# #4-step model:
# import pandas
# import geopandas
# import json
# import math
# from haversine import haversine
# from ipfn import ipfn
# import networkx
# from matplotlib import pyplot
# from matplotlib import patheffects


In [549]:
# =============
# CONFIGURATION
# =============

# URL of the file to download
url = "https://services1.arcgis.com/Ug5xGQbHsD8zuZzM/arcgis/rest/services/ACS_2022_Geographic_boundaries/FeatureServer/replicafilescache/ACS_2022_Geographic_boundaries_-7361254879251475346.zip"

# Filename to save as
filename = "ACS_2022_Geographic_boundaries_-7361254879251475346.zip"
# Extracted folder name
extracted_name = filename.rsplit('.', 1)[0]

# Format: ([GEOID], [IS_IN_BELTLINE])
GEOIDS = [
    ('BELTLINE01', True),
    ('BELTLINE02', True),
    ('BELTLINE03', True),
    ('BELTLINE04', True),
    ('BELTLINE05', True),
    ('BELTLINE06', True),
    ('BELTLINE07', True),
    ('BELTLINE08', True),
    ('BELTLINE09', True),
    ('BELTLINE10', True),
    ('HS6443070', False),   # Gresham Park
    ('HS6442055', False),   # Druid Hills
    ('1322052', False),     # Decatur city
    ('1325720', False),     # East Point
    ('HS6331054', False),   # Campbell HS
    ('1310944', False),     # BrookHaven City
    ('urmom', False),
    ('1304000', False)      # Atlanta City - KEEP AS LAST ELEMENT (for graphing purposes)
    ]

EPSILON = 1e-3

In [550]:
# DOWNLOAD AND EXTRACT FILE METHOD

cwd = Path.cwd()

def download_file(url, filename):
    # Use a 'data' subfolder in the current working directory
    data_dir = cwd / "data"
    data_dir.mkdir(exist_ok=True)  # Create the 'data' directory if it doesn't exist
    file_path = data_dir / filename

    # Check if file already exists
    if file_path.exists():
        print(f"{filename} already exists in {data_dir}. Skipping download.")
        return file_path
        
    else:
        # Make the request
        print(f"Downloading {filename} to {data_dir}...")
        response = requests.get(url, stream=True)
        
        # Check if the request was successful
        if response.status_code == 200:
            content_type = response.headers.get('Content-Type', '')
            if 'application/zip' not in content_type and 'application/octet-stream' not in content_type:
                print(f"Unexpected Content-Type: {content_type}. The downloaded file may not be a ZIP archive.")
                return None
            
            # Get the total file size
            total_size = int(response.headers.get('content-length', 0))
            # Open the file and use tqdm for the progress bar
            with file_path.open('wb') as file, tqdm(
                desc=filename,
                total=total_size,
                unit='iB',
                unit_scale=True,
                unit_divisor=1024,
            ) as progress_bar:
                for data in response.iter_content(chunk_size=1024):
                    size = file.write(data)
                    progress_bar.update(size)
            print(f"Successfully downloaded {filename}")
            return file_path  # Return the file path after successful download
                
        else:
            print(f"Failed to download {filename}. Status code: {response.status_code}")
            return None
        
def extract_file(file_path):
    if not zipfile.is_zipfile(file_path):
        print(f"The file {file_path} is not a valid ZIP archive. Extraction aborted.")
        return
    
    data_dir = file_path.parent
    extract_path = data_dir / extracted_name # Get path to unzipped extract folder
    
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        
        # Extract the ZIP file
        extract_path.mkdir(exist_ok=True) # Create the extraction folder if it doesn't exist
        print(f"Extracting {file_path.name} to {extract_path}...")
        
        # Get the total number of files in the ZIP
        total_files = len(zip_ref.infolist())
        # Use tqdm for the extraction progress bar
        for file in tqdm(zip_ref.infolist(), desc="Extracting", total=total_files):
            zip_ref.extract(file, extract_path)
    
    print(f"Successfully extracted {file_path.name} to {extract_path}")
        
# Download file
file_path = download_file(url, filename)

#Extract file
if file_path:
    extract_file(file_path)

ACS_2022_Geographic_boundaries_-7361254879251475346.zip already exists in c:\Users\kmmat\OneDrive\Desktop\VIP\24Fa-MPONC\modeling_processes_of_neighborhood_change_new\data. Skipping download.
Extracting ACS_2022_Geographic_boundaries_-7361254879251475346.zip to c:\Users\kmmat\OneDrive\Desktop\VIP\24Fa-MPONC\modeling_processes_of_neighborhood_change_new\data\ACS_2022_Geographic_boundaries_-7361254879251475346...


Extracting: 100%|██████████| 6/6 [00:00<00:00, 14.56it/s]

Successfully extracted ACS_2022_Geographic_boundaries_-7361254879251475346.zip to c:\Users\kmmat\OneDrive\Desktop\VIP\24Fa-MPONC\modeling_processes_of_neighborhood_change_new\data\ACS_2022_Geographic_boundaries_-7361254879251475346


In [551]:
# ===========================
# GeoDataFrame FILE SETUP/MANIPULATION
# ===========================

# Establish/create paths
figures_folder = cwd / "figures"
figures_folder.mkdir(parents=True, exist_ok=True)

extracted_folder = cwd / 'data' / extracted_name # Path to extraction folder
graph_file = cwd / 'data/ACS_2022_Geographic_boundaries_-7361254879251475346/atlanta.pkl' # Path to pkl file

# Generate OSMnx graph
GA_gdf = gpd.read_file(extracted_folder / 'Geographic_boundaries,_ACS_2022.shp')
    
# Simplify geometries
GA_gdf['geometry'] = GA_gdf['geometry'].simplify(tolerance=0.001, preserve_topology=True)    

# Add 'Sqkm' area column:
GA_gdf = GA_gdf.to_crs(epsg=32616) # Update CRS for area calculations
GA_gdf['Sqkm'] = GA_gdf['geometry'].area / 1e+6 

# Update CRS for general calculations (ex. amenity retrieval)
GA_gdf = GA_gdf.to_crs(epsg=4326)

In [552]:
# ====================
# GRAPH INITIALIZATION
# ====================

valid_GEOIDS = set(GA_gdf['GEOID']) # Set of valid GEOID's
used_GEOIDS = [] # Array to store (valid) GEOID's being used for simulation
invalid_GEOIDS = [] # Array to store invalid GEOID's
gdf_sub_array = [] # Array containing each GEOID's gdf

def create_graph():
    for geoid, _ in tqdm(GEOIDS, desc="Filtering for GEOIDs", unit="geoid"):
        if geoid in valid_GEOIDS:
            # Filter GA_gdf for the current GEOID
            gdf_sub = GA_gdf[GA_gdf['GEOID'] == geoid]
            gdf_sub_array.append(gdf_sub)
            used_GEOIDS.append(geoid)
        else:
            invalid_GEOIDS.append(geoid)
            
    # Concatenate GEOID regions
    if (gdf_sub_array):
        gdf_combined = pd.concat(gdf_sub_array, ignore_index=True)
    else:
        raise ValueError("No valid GEOID's found.")
    
    # Combines all polygons within GEOID regions
    combined_shape = gdf_combined.geometry.union_all() 
    
    print("Generating graph from OSMnx...")
    
    g = ox.graph_from_polygon(combined_shape, network_type='drive', simplify=True) #Roadmap of Atlanta - networkx.MultiDiGraph
    g = g.subgraph(max(nx.strongly_connected_components(g), key=len)).copy() #Ensures all nodes are connected
    g = nx.convert_node_labels_to_integers(g) #Converts nodes to integers
    
    print("Graph generated.")
    print (f"Invalid GEOID's: {invalid_GEOIDS}")
    return g, used_GEOIDS
   
def save_graph(g, used_GEOIDS, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump({'graph': g, 'GEOIDS': used_GEOIDS}, file)
    print(f"Graph saved to {file_path}.")
    
def load_graph(file_path):
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    g = data['graph']
    saved_GEOIDS = data['GEOIDS']
    print(f"Graph loaded from {file_path}.")
    return g, saved_GEOIDS


# ====================
# MAIN EXECUTION LOGIC
# ====================
if os.path.exists(graph_file):
    # Load existing graph and its GEOIDS
    g, saved_GEOIDS = load_graph(graph_file)
    
    # Compare saved_GEOIDS with current GEOIDS
    if saved_GEOIDS != used_GEOIDS:
        print("GEOIDs have changed. Recreating the graph...")
        g, used_geoids = create_graph()
        save_graph(g, used_GEOIDS, graph_file)
    else:
        print("Current GEOIDs match the saved graph. Using the existing graph.")
        
else:
    # Create new graph
    print ("Creating graph for the first time.")
    g, used_GEOIDS = create_graph()
    save_graph(g, used_GEOIDS, graph_file)

Graph loaded from c:\Users\kmmat\OneDrive\Desktop\VIP\24Fa-MPONC\modeling_processes_of_neighborhood_change_new\data\ACS_2022_Geographic_boundaries_-7361254879251475346\atlanta.pkl.
GEOIDs have changed. Recreating the graph...


Filtering for GEOIDs: 100%|██████████| 18/18 [00:00<00:00, 1747.91geoid/s]

Generating graph from OSMnx...


Graph generated.
Invalid GEOID's: ['urmom']
Graph saved to c:\Users\kmmat\OneDrive\Desktop\VIP\24Fa-MPONC\modeling_processes_of_neighborhood_change_new\data\ACS_2022_Geographic_boundaries_-7361254879251475346\atlanta.pkl.


In [553]:
# =======================
# CENTROID INITIALIZATION
# =======================

#CENTROIDS
centroids = []
# tuple format: (longitude, latitude, region_name, is_beltline)

GEOID_info = {geoid: is_beltline for geoid, is_beltline in GEOIDS}

#CENTROID INITIALIZATION FROIM GEOIDS
for geoid in tqdm(used_GEOIDS[:-1], desc="Initializing Centroids", unit="centroid"):
    
    # Default is_beltline to False if it's not in tuple
    is_beltline = GEOID_info.get(geoid, False)
    
    # Fetch GEOID instance from GA_gdf
    gdf_sub = GA_gdf[GA_gdf['GEOID'] == geoid]
    
    # Combined geometry of all geometries in gdf_sub
    combined_geometry = gdf_sub.geometry.union_all()
    
    # Initialize centroid with coordinates
    centroid = combined_geometry.centroid
    centroids.append((centroid.x, centroid.y, gdf_sub['Name'].iloc[0], is_beltline))
    
print(centroids)

Initializing Centroids: 100%|██████████| 16/16 [00:00<00:00, 976.31centroid/s]

[(-84.4300872841787, 33.73337737454873, 'RDA/Lee/Cascade/Enota Park', True), (-84.39998999929675, 33.72073292110259, 'Heritage Communities of South Atlanta', True), (-84.36588299775734, 33.72731079399028, 'Boulevard/McDonough/Boulevard Crossing', True), (-84.35800021912941, 33.747177151680226, 'Memorial/Glenwood/Grant Park', True), (-84.36239654093053, 33.76309351383761, 'Freedom Parkway/Fourth Ward Park', True), (-84.37058556519439, 33.79122332709902, 'Virginia-Highland/Ansley/Piedmont Park', True), (-84.38641032897236, 33.81384973688155, 'Peachtree/Collier/Tanyard Creek', True), (-84.41820098662457, 33.79384461458277, 'Howell/Northside/Water Works', True), (-84.4321553136524, 33.779854997692205, 'West Marietta/Howell/Westside Park', True), (-84.42538511491587, 33.758754894790236, 'Hollowell/Boone/Maddox Park', True), (-84.31414331114354, 33.70719989012727, 'McNair, Ronald E. HS', False), (-84.29437743123609, 33.796223572057436, 'Druid Hills HS', False), (-84.29617089315472, 33.771027

In [554]:
# ================================================
# AMENITY DENSITY & CENTROID DISTANCE CALCULATIONS
# ================================================

# Amenity Density Calculation
def compute_amts_dens():
    
    # [AMENITY FILTER]
    tags = {'highway':'bus_stop'} #TODO: I don't think number of bus stops is accurate

    # Initialize empty arrays
    amenities = np.zeros(len(used_GEOIDS) - 1)
    areas_sqkm = np.zeros(len(used_GEOIDS) - 1)
    
    # Iterate over each GEOID
    for ID, geoid in enumerate(tqdm(used_GEOIDS[:-1], desc="Computing Amenity Densities", unit="GEOID")):
    
        # Extract polygon of current GEOID
        polygon = GA_gdf.loc[GA_gdf['GEOID'] == geoid, 'geometry'].union_all()
        
        # Collect amenities
        try:
            amenities[ID] = len(ox.features_from_polygon(polygon, tags))
            
        except: # No bus stops
            amenities[ID] = 0
        
        # Sum area current GEOID
        areas_sqkm[ID] = GA_gdf.loc[GA_gdf['GEOID'] == geoid, 'Sqkm'].sum()
        
        #For troubleshooting:
        tqdm.write(f"{geoid}: {areas_sqkm[ID]:.2f} sq km, {amenities[ID]:.2f} amenities")
        

    # Amenity density (amenities / square kilometer)
    amts_dens = amenities / areas_sqkm
    
    # Normalize densities
    if np.max(amts_dens) > 0:
        amts_dens /= np.max(amts_dens)
    
    return amts_dens


# Distances Between Centroids Calculation
def compute_centroid_distances():
    n = len(centroids)
    
    # Map centroids to nearest node
    centroid_nodes = [ox.nearest_nodes(g, c[1], c[0]) for c in centroids]
    # 2D matrix of centroid-to-centroid distance
    distance_matrix = np.zeros((n, n))
    
    # Loop through each pair of nodes
    for i in tqdm(range(n), desc="Computing centroid distances"):
        source_node = centroid_nodes[i]
        for j in range(i, n):
            target_node = centroid_nodes[j]
            
            # Retrieve distance between source and target centroid nodes
            if (i == j):
                distance = 0
            else :
                distance = nx.shortest_path_length(g, source=source_node, target=target_node, weight='length')
                distance_matrix[i][j] = distance
                distance_matrix[j][i] = distance #symmetrix matrix
    
    # Normalize and return
    if distance_matrix.max() > 0:
        distance_matrix  /= distance_matrix.max()
    return distance_matrix


#compute density of amenities
amts_dens = compute_amts_dens()
        
# Initialize dictionary of distances between centroids
centroid_distances = compute_centroid_distances()

Computing Amenity Densities:  12%|█▎        | 2/16 [00:00<00:00, 17.30GEOID/s]

BELTLINE01: 8.11 sq km, 5.00 amenities
BELTLINE02: 7.35 sq km, 0.00 amenities


Computing Amenity Densities:  12%|█▎        | 2/16 [00:00<00:00, 17.30GEOID/s]

BELTLINE03: 5.05 sq km, 5.00 amenities


Computing Amenity Densities:  25%|██▌       | 4/16 [00:00<00:00, 18.21GEOID/s]

...

Computing Amenity Densities:  88%|████████▊ | 14/16 [00:00<00:00, 15.35GEOID/s]

1325720: 38.10 sq km, 13.00 amenities


Computing Amenity Densities:  88%|████████▊ | 14/16 [00:00<00:00, 15.35GEOID/s]

HS6331054: 85.21 sq km, 67.00 amenities
1310944: 32.09 sq km, 6.00 amenities


Computing centroid distances: 100%|██████████| 16/16 [00:00<00:00, 41070.30it/s]


In [555]:
class Agent:
    
    def __init__(self, i, dow, city, alpha=0.5):

        ''' 
        Initialize an Agent instance
        Parameters:
        - i (int): Agent identifier.
        - dow (float): Endowment value.
        - city (City): Reference to the City instance.
        - alpha (float): Weighting factor for transit access vs. community value.
        '''
        self.i = i
        self.dow = dow
        self.city = city 
        self.alpha = alpha

        self.weights = None
        self.probabilities = None # Probability to go to each centroid
        self.tot_probabilities = None
        self.avg_probabilities = None
        self.u = None

        self.reset()
        
    # Create hash identifier
    def __hash__(self):
        return hash(self.i)

    def __eq__(self, other): 
        return self.i == other.i

    '''
    - Assign starting centroid
    - Reset centroid weights/probabilities
    '''
    def reset(self):
        
        # Initialize weights
        self.weights = np.ones(len(centroids))
        
        # Initialize probabilities
        self.probabilities = np.array(self.weights / self.weights.sum())
        
        # Initialize tot_probabilities
        self.tot_probabilities = self.probabilities
        
        # Initialize current node - Initialize starting position at random node (based on weights)
        self.u = np.random.choice(self.city.n, p=self.probabilities) 
        
        # Adds self to the current node's set of inhabitants in inh_array
        self.city.inh_array[self.u].add(self)

# ACTION METHOD
    def act(self): 
        # Leave node
        self.city.inh_array[self.u].remove(self)
    
        # Choose another node
        self.u = np.random.choice(self.city.n, p=self.probabilities) 
    
        # Join node
        self.city.inh_array[self.u].add(self)
        
# LEARN METHOD
    def learn(self):
        costs = self.cost_vector()
        self.weights *= (1 - EPSILON * costs) # array of weights, based on COST
        self.probabilities = self.weights / np.sum(self.weights) #normalize
        self.tot_probabilities += self.probabilities
        
    def cost_vector(self):
        dow_thr = self.city.dow_thr_array
        
        aff = (self.dow >= dow_thr).astype(float) # Affordability: Is the agent's endowment >= endowment threshold?
        loc = centroid_distances[self.u, :] # Location cost: Distances from centroid to all other centroids
        cmt = np.exp(- self.alpha * np.abs(self.dow - self.city.cmt_array)) # Community cost
        acc = np.exp(- (1 - self.alpha) * amts_dens) # Accessibility cost
        upk = self.city.upk_array # Upkeep cost
        beltline = self.city.beltline_array # Beltline cost
        
        cost = 1 - aff * upk * beltline * loc * cmt * acc
        return cost
        

In [556]:
# # ===============
# # FOUR-STEP MODEL
# # ===============

# # SUPPLY data (transportation network):
# url = 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/State_County/MapServer/37/query?where=state%3D06&f=geojson'
# r = requests.get(url)
# zones = geopandas.GeoDataFrame.from_features(r.json()['features'])
# centroidFunction = lambda row: (row['geometry'].centroid.y, row['geometry'].centroid.x)
# zones['centroid'] = zones.apply(centroidFunction, axis=1)

# # DEMAND data (transportation network users):
# url = 'http://api.census.gov/data/2015/acs5/profile?get=NAME,DP03_0018E&for=county&in=state:06'
# r = requests.get(url)
# Production = pandas.DataFrame(r.json()[1:], columns = r.json()[0], dtype='int')
# nameSplit = lambda x: x.split(',')[0]
# Production['NAME'] = Production['NAME'].apply(nameSplit)
# zones = pandas.merge(zones, Production)
# zones['Production'] = zones['DP03_0018E']

# def getEmployment(state, county):
#     prefix = 'EN'
#     seasonal_adjustment = 'U'
#     area = format(state, "02d") + format(county, "03d")
#     data_type = '1'
#     size = '0'
#     ownership = '0'
#     industry = '10'
#     seriesid = prefix + seasonal_adjustment + area + data_type + size + ownership + industry
#     headers = {'Content-type': 'application/json'}
#     data = json.dumps({"seriesid": [seriesid],"startyear":"2015", "endyear":"2015", "registrationKey": ""})
#     p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
#     employment = p.json()['Results']['series'][0]['data'][0]['value']
#     return(employment)

# employment = lambda row: int(getEmployment(row['state'], row['county']))
# zones['Attraction'] = zones.transpose().apply(employment)
# zones['Production'] = zones['Production'] * zones.sum()['Attraction'] / zones.sum()['Production']
# zones.index = zones.NAME
# zones.sort_index(inplace=True)

# # TRIP DISTRIBUTION:
# def costFunction(zones, zone1, zone2, beta):
#     cost = math.exp(-beta * haversine(zones[zone1]['centroid'], zones[zone2]['centroid']))
#     return(cost)

# def costMatrixGenerator(zones, costFunction, beta):
#     originList = []
#     for originZone in zones:
#         destinationList = []
#         for destinationZone in zones:
#             destinationList.append(costFunction(zones, originZone, destinationZone, beta))
#         originList.append(destinationList)
#     return(pandas.DataFrame(originList, index=zones.columns, columns=zones.columns))

# def tripDistribution(tripGeneration, costMatrix):
#     costMatrix['ozone'] = costMatrix.columns
#     costMatrix = costMatrix.melt(id_vars=['ozone'])
#     costMatrix.columns = ['ozone', 'dzone', 'total']
#     production = tripGeneration['Production']
#     production.index.name = 'ozone'
#     attraction = tripGeneration['Attraction']
#     attraction.index.name = 'dzone'
#     aggregates = [production, attraction]
#     dimensions = [['ozone'], ['dzone']]
#     IPF = ipfn.ipfn(costMatrix, aggregates, dimensions)
#     trips = IPF.iteration()
#     return(trips.pivot(index='ozone', columns='dzone', values='total'))

# beta = 0.01
# costMatrix = costMatrixGenerator(zones.transpose(), costFunction, beta)
# trips = tripDistribution(zones, costMatrix)

# # MODE CHOICE:
# def modeChoiceFunction(zones, zone1, zone2, modes):
#     distance = haversine(zones[zone1]['centroid'], zones[zone2]['centroid'])
#     probability = {}
#     total = 0.0
#     for mode in modes:
#         total = total + math.exp(modes[mode] * distance)
#     for mode in modes:
#         probability[mode] = math.exp(modes[mode] * distance) / total
#     return(probability)

# def probabilityMatrixGenerator (zones, modeChoiceFunction, modes):
#     probabilityMatrix = {}
#     for mode in modes:
#         originList = []
#         for originZone in zones:
#             destinationList = []
#             for destinationZone in zones:
#                 destinationList.append(modeChoiceFunction(zones, originZone, destinationZone, modes)[mode])
#             originList.append(destinationList)
#         probabilityMatrix[mode] = pandas.DataFrame(originList, index=zones.columns, columns=zones.columns)
#     return(probabilityMatrix)

# modes = {'walking': .05, 'cycling': .05, 'driving': .05}
# probabilityMatrix = probabilityMatrixGenerator(zones.transpose(), modeChoiceFunction, modes)
# drivingTrips = trips * probabilityMatrix['driving']

# #ROUTE ASSIGNMENT:
# def routeAssignment(zones, trips):
#     G = networkx.Graph()
#     G.add_nodes_from(zones.columns)
#     for zone1 in zones:
#         for zone2 in zones:
#             if zones[zone1]['geometry'].touches(zones[zone2]['geometry']):
#                 G.add_edge(zone1, zone2, distance = haversine(zones[zone1]['centroid'], zones[zone2]['centroid']), volume=0.0)
#     for origin in trips:
#         for destination in trips:
#             path = networkx.shortest_path(G, origin, destination)
#             for i in range(len(path) - 1):
#                 G[path[i]][path[i + 1]]['volume'] = G[path[i]][path[i + 1]]['volume'] + trips[zone1][zone2]
#     return(G)

# def visualize(G, zones):
#     fig = pyplot.figure(1, figsize=(10, 10), dpi=90)
#     ax = fig.add_subplot(111)
#     zonesT = zones.transpose()
#     zonesT.plot(ax = ax)
#     for i, row in zones.transpose().iterrows():
#         text = pyplot.annotate(s=row['NAME'], xy=((row['centroid'][1], row['centroid'][0])), horizontalalignment='center', fontsize=6)
#         text.set_path_effects([patheffects.Stroke(linewidth=3, foreground='white'), patheffects.Normal()])
#     for zone1 in G.edge:
#         for zone2 in G.edge[zone1]:
#             volume = G.edge[zone1][zone2]['volume']
#             x = [zones[zone1]['centroid'][1], zones[zone2]['centroid'][1]]
#             y = [zones[zone1]['centroid'][0], zones[zone2]['centroid'][0]]
#             ax.plot(x, y, color='#444444', linewidth=volume/10000, solid_capstyle='round', zorder=1)
#     pyplot.show(block=False)

# G = routeAssignment(zones.transpose(), drivingTrips)
# visualize(G, zones.transpose())

# # TO PLAY AROUND WITH PARAMETERS (LIST OF PARAMETERS:)
# # Trip Distribution
#     #beta = 0.01
#     #costMatrix = costMatrixGenerator(zones.transpose(), costFunction, beta)
#     #trips = tripDistribution(zones, costMatrix)
# # Mode Choice
#     #modes = {'walking': .05, 'cycling': .05, 'driving': .05}
#     #probabilityMatrix = probabilityMatrixGenerator(zones.transpose(), modeChoiceFunction, modes)
#     #drivingTrips = trips * probabilityMatrix['driving']
# # Route Assignment
#     #G = routeAssignment(zones.transpose(), drivingTrips)
#     #visualize(G, zones.transpose())

In [557]:
class City:

    # CONSTRUCTOR
    def __init__(self, centroids, g, rho=2): #default rho (house capacity) == 2
        '''
        Initialize a City instance.
        '''
        self.rho = rho #house capacity
        self.centroids = centroids #centroids list
        self.g = g #OSMnx map
        self.n = len(centroids)
        
        # STORE ATTRIBUTES OF ALL CENTROIDS 
        self.lat_array = np.array([lat for lat, _, _, _ in centroids]) # Latitude
        self.lon_array = np.array([lon for _, lon, _, _ in centroids]) # Longitude
        self.beltline_array = np.array([beltline for _, _, _, beltline in centroids], dtype=bool).astype(float) # In Beltline?
        self.name_array = [name for _, _, name, _ in centroids] # Centroid region name
        
        self.inh_array = [set() for _ in range(self.n)] # Array of sets - each set contains Agent inhabitants
        self.dow_thr_array = np.zeros(self.n) # Endowment threshold
        self.upk_array = np.zeros(self.n, dtype=bool) # Upkeep score
        self.cmt_array = np.zeros(self.n) # Community score
        
        self.pop_hist = [[] for _ in range(self.n)]  # Population history - list of lists 
        self.cmt_hist = [[] for _ in range(self.n)]  # Community score history - list of lists 
        
        self.node_array = np.array([ox.nearest_nodes(self.g, lon, lat) for lon, lat in zip(self.lon_array, self.lat_array)])
        
        # # Create a dictionary with each centroid's attributes
        # self.centroidDict = {} 
        # for ID, (lat, lon, name, beltline) in enumerate(self.centroids): #Iterate through each centroid
        #     self.centroidDict[ID] = {
        #         'lat': lat, # Latitude
        #         'lon': lon, # Longitude
        #         'name': name, # Housing name (region)
        #         'beltline': beltline,  # Is it in the Beltline?
                
        #         'inh': set(),  # Set containing all Agent inhabitants
        #         'dow_thr': 0.0,  # Endowment threshold initialized to 0
        #         'upk': False,  # Upkeep score
        #         'cmt': 0.0,  # Community score
        #         'pop_hist': [], # Population history
        #         'cmt_hist': [],  # Community history
                
        #         'node': ox.nearest_nodes(self.g, lon, lat)
        #     }
            

    # Set agents and their endowments
    def set_agts(self, agts):
        self.agts = agts #list of agents
        self.agt_dows = np.array([a.dow for a in self.agts]) #array of agent endowments

    # Update each node (cmt score, population)
    def update(self):   
        for ID in range(self.n): # For each centroid

            inhabitants = self.inh_array[ID]
            
            pop = len(inhabitants)
            
            # Update population history
            self.pop_hist[ID].append(pop)
            
            if pop > 0: # Inhabited
                #UPDATE COMMUNITY SCORE (avg inhabitant dows, weighted by distance to other centroids)
                inhabitant_dows = np.array([a.dow for a in self.inh_array[ID]])  # Array of endowments of node's inhabitants
                distances = centroid_distances[ID, [a.u for a in self.inh_array[ID]]]
                weights = (1 - distances) ** 2
                
                # Update Community history (average endowment)
                cmt = np.average(inhabitant_dows, weights=weights) 
                
                # Establish endowment threshold
                if pop < self.rho:
                    self.dow_thr_array[ID] = 0.0
                else:
                    self.dow_thr_array[ID] = np.partition(inhabitant_dows, -self.rho)[-self.rho]
                self.upk_array[ID] = True
                
            else: # If uninhabited
                self.dow_thr_array[ID] = 0.0
                self.upk_array[ID] = False
                cmt = 0.0

            # Update Community history (average endowment)
            self.cmt_hist[ID].append(cmt)
        
        
    # =============
    # PLOTTING CODE
    # =============
    def plot(self, cmap='YlOrRd', figkey='city', graph=None):
        fig, ax = plt.subplots(figsize=(10, 10))
    
        if graph:
            ox.plot_graph(graph, ax=ax, node_color='black', node_size=10, edge_color='gray', edge_linewidth=1, show=False, close=False)
    
        # Prepare agent data
        agent_lats = np.array([self.lat_array[agent.u] for agent in self.agts])
        agent_lons = np.array([self.lon_array[agent.u] for agent in self.agts])
        agent_wealths = np.array([agent.dow for agent in self.agts])
    
        # Population density heatmap
        heatmap, xedges, yedges = np.histogram2d(agent_lons, agent_lats, bins=30)
        extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
        ax.imshow(heatmap.T, extent=extent, origin='lower', cmap=cmap, alpha=0.5)
    
        # Plot agents with wealth-based marker sizes
        norm = Normalize(vmin=agent_wealths.min(), vmax=agent_wealths.max())
        marker_sizes = [50 + 150 * norm(agent_wealths)]
        sc = ax.scatter(agent_lons, agent_lats, c=agent_wealths, s=marker_sizes, cmap='coolwarm', alpha=0.7, edgecolor='red')
    
        # Plot centroids locations (this comes after the graph to make sure they are visible on top)
        
        colors = np.where(self.beltline_array, 'yellow', 'white')
        ax.scatter(self.lon_array, self.lat_array, color=colors, s=100, alpha=0.7, edgecolor='black')
            
        for ID in range(self.n):    
            lon = self.lon_array[ID]
            lat = self.lat_array[ID]
            # Display inhabitant populations at each node:
            inhabitants = len(self.inh_array[ID])
            ax.text(lon, lat, str(inhabitants), fontsize=9, ha='center', va='center', color='black')

    
        # Add color bar for wealth
        cbar = plt.colorbar(sc, ax=ax, orientation='vertical', label='Wealth (dow)')
    
        # Labels and title
        ax.set_title(f"City Visualization: {figkey}")
        ax.set_xlabel("Longitude")
        ax.set_ylabel("Latitude")
    
        # Legend
        ax.scatter([], [], c='yellow', s=100, label='Beltline Housing')
        ax.scatter([], [], c='white', s=100, label='Non-Beltline Housing')
        ax.scatter([], [], c='red', s=100, label='Agents')
        ax.legend(loc='upper right')
    
        plt.tight_layout()
        plt.savefig(f'./figures/{figkey}.pdf', format='pdf', bbox_inches='tight')
        plt.close()

In [558]:
# ==========
# SIMULATION 
# ==========

# =========================
# PRE-DETERMINED PARAMETERS
# =========================
rho_l = [4] #1, 2, 4, 8 (for each iteration) rho-house capacity
alpha_l = [0.25] #0.25, 0.75 (for each iteration) lambda - centroid proximity vs. community value
t_max_l = [5000] #5000, 10000, 15000, 20000 (for each iteration) timesteps
tau = 0.5 # inequality factor in Lorentz curve
num_agents = 100

# RUN SIMULATION?
run_experiments = True

# PLOT SIMULATION?
plot_cities = True

cty_key = 'Atlanta'

# ===============
# SIMULATION CODE
# ===============
# =========================
if run_experiments:
    for rho in rho_l:
        for alpha in alpha_l:

            np.random.seed(0)

            city = City(centroids, g, rho=rho)
            agt_dows = np.diff([1 - (1 - x) ** tau for x in np.linspace(0, 1, num_agents + 1)]) 
            agts = [Agent(i, dow, city, alpha=alpha) for i, dow in enumerate(agt_dows)]

            city.set_agts(agts)
            city.update()

            for t in range(max(t_max_l)):
                print('t: {0}'.format(t))
                for a in agts:
                    a.act()
                city.update()
                for a in agts:
                    a.learn()
                
                if t + 1 in t_max_l:

                    for a in city.agts:
                        a.avg_probabilities = a.tot_probabilities / (t + 1)

                    with open(Path(cwd / 'data/{0}_{1}_{2}_{3}.pkl'.format(cty_key, rho, alpha, t + 1)), 'wb') as file:
                        pickle.dump(city, file)

if plot_cities:
    for rho in rho_l:
        for alpha in alpha_l:
            for t_max in t_max_l:
                with open(Path(cwd / 'data/{0}_{1}_{2}_{3}.pkl'.format(cty_key, rho, alpha, t_max)), 'rb') as file:
                    city = pickle.load(file)
                cmap = 'YlOrRd'
                figkey = '{0}_{1}_{2}_{3}'.format(cty_key, rho, alpha, t_max)
                city.plot(cmap=cmap, figkey=figkey, graph=g)

t: 0
t: 1
t: 2
t: 3
t: 4
t: 5
t: 6
t: 7
t: 8
t: 9
t: 10
t: 11
t: 12
t: 13
t: 14
t: 15
t: 16
t: 17
t: 18
t: 19
t: 20
t: 21
t: 22
t: 23
t: 24
t: 25
t: 26
t: 27
t: 28
t: 29
t: 30
t: 31
t: 32
t: 33
t: 34
t: 35
t: 36
t: 37
t: 38
t: 39
t: 40
t: 41
t: 42
t: 43
t: 44
t: 45
t: 46
t: 47
t: 48
t: 49
t: 50
t: 51
t: 52
t: 53
t: 54
t: 55
t: 56
t: 57
t: 58
t: 59
t: 60
t: 61
t: 62
t: 63
t: 64
t: 65
t: 66
t: 67
t: 68
t: 69
t: 70
t: 71
t: 72
t: 73
t: 74
t: 75
t: 76
t: 77
t: 78
t: 79
t: 80
t: 81
t: 82
t: 83
t: 84
t: 85
t: 86
t: 87
t: 88
t: 89
t: 90
t: 91
t: 92
t: 93
t: 94
t: 95
t: 96
t: 97
t: 98
t: 99
t: 100
t: 101
t: 102
t: 103
t: 104
t: 105
t: 106
t: 107
t: 108
t: 109
t: 110
t: 111
t: 112
t: 113
t: 114
t: 115
t: 116
t: 117
t: 118
t: 119
t: 120
t: 121
t: 122
t: 123
t: 124
t: 125
t: 126
t: 127
t: 128
t: 129
t: 130
t: 131
t: 132
t: 133
t: 134
t: 135
t: 136
t: 137
t: 138
t: 139
t: 140
t: 141
t: 142
t: 143
t: 144
t: 145
t: 146
t: 147
t: 148
t: 149
t: 150
t: 151
t: 152
t: 153
t: 154
t: 155
t: 156
t: 157
t: 1

C:\Users\kmmat\AppData\Local\Temp\ipykernel_17212\4142455164.py:141: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


ValueError: Image size of 225239x225357 pixels is too large. It must be less than 2^16 in each direction.

ValueError: Image size of 225239x225357 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 1000x1000 with 2 Axes>